In [ ]:
import torch
import torch.nn as nn
import numpy as np
from scipy.stats import halfnorm
import scipy.stats as stats
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy.optimize import minimize
import statsmodels.api as sm
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from concurrent.futures import ProcessPoolExecutor
import seaborn as sns
from tqdm import tqdm

# Stochastic Frontier Model类
class StochasticFrontierModel(nn.Module):
    def __init__(self, num_features):
        super().__init__()
        self.beta = nn.Parameter(torch.tensor(num_features * [1.0], dtype=torch.float32))
        self.log_sigma2 = nn.Parameter(torch.tensor(0.0))
        self.log_lambda0 = nn.Parameter(torch.tensor(0.0))

    def predict(self, x):
        predictions = torch.matmul(x, self.beta)
        return predictions

    def forward(self, x, y):
        sigma2 = torch.exp(self.log_sigma2)
        lambda0 = torch.exp(self.log_lambda0)
        sigma = torch.sqrt(sigma2)
        epsilon = y - torch.sum(x * self.beta, dim=1)

        term1 = x.shape[0] * self.log_sigma2 / 2
        term2 = -torch.sum(torch.log(torch.distributions.normal.Normal(0, 1).cdf(-epsilon * lambda0 / sigma) + 1e-10))
        term3 = torch.sum(epsilon**2) / (2 * sigma2)

        return (term1 + term2 + term3) / x.shape[0]

class StochasticFrontierAnalysis:
    def __init__(self, num_features, beta, sigma_u, sigma_v):
        self.num_features = num_features
        self.beta = beta
        self.sigma_u = sigma_u
        self.sigma_v = sigma_v

    def standardize_data(self, x):
        mean = np.mean(x[:, 1:], axis=0)
        std = np.std(x[:, 1:], axis=0)
        x_standardized = np.column_stack([x[:, 0], (x[:, 1:] - mean) / std])
        return x_standardized, np.insert(mean, 0, 0), np.insert(std, 0, 1)

    def generate_data(self, N):
        x1 = np.random.uniform(0, 1, N)
        x_random = np.random.normal(0, 1, (N, self.num_features - 1))
        x = np.hstack([x1.reshape(-1, 1), x_random])
        v = np.random.normal(0, self.sigma_v, N)
        u = stats.halfnorm.rvs(loc=0, scale=self.sigma_u, size=N)
        y = x.dot(self.beta) + v - u
        return x, y

    

    def train_model_without_private(self, x, y, num_iters=3000, constraint=100, minibatch_size=50):
        x, mean, std = self.standardize_data(x)
        x = torch.tensor(x, dtype=torch.float32)
        y = torch.tensor(y, dtype=torch.float32)
        model = StochasticFrontierModel(num_features=self.num_features)
        losses = []
        gradients = []
        parameters = []

        for i in tqdm(range(1, num_iters + 1), desc="Training without privacy"):
            minibatch_indices = np.random.choice(x.shape[0], minibatch_size, replace=True)
            minibatch_x = x[minibatch_indices]
            minibatch_y = y[minibatch_indices]
            loss = model(minibatch_x, minibatch_y)
            loss.backward()
            gradient = torch.cat([p.grad.flatten() for p in model.parameters()]).detach().numpy()
            pos_alphas = self.compute_alpha(constraint, gradient, model)
            neg_alphas = self.compute_alpha(-constraint, gradient, model)
            alphas = pos_alphas + neg_alphas
            min_alpha, size, corner_num = min(alphas, key=lambda x: x[0])
            corner = np.zeros(sum(p.numel() for p in model.parameters()))
            corner[corner_num] = size
            mu = 2 / (i + 2)
            with torch.no_grad():
                index = 0
                for p in model.parameters():
                    numel = p.numel()
                    p_flat = p.view(-1)
                    p_flat.copy_((1 - mu) * p_flat + mu * torch.tensor(corner[index:index+numel], dtype=torch.float32))
                    index += numel
            losses.append(loss.item())
            gradients.append(np.linalg.norm(gradient))
            parameters.append(model.state_dict().copy())
            model.zero_grad()

        min_loss_index = np.argmin(losses)
        model.load_state_dict(parameters[min_loss_index])

        return model, mean, std, losses[-1], self.calculate_mse(model, x, y)

    def train_model_private(self, x, y, num_iters=3000, constraint=100, minibatch_size=50, lipschitz=1, epsilon=0.1, delta=1e-5):
        x, mean, std = self.standardize_data(x)
        x = torch.tensor(x, dtype=torch.float32)
        y = torch.tensor(y, dtype=torch.float32)
        model = StochasticFrontierModel(num_features=self.num_features)
        n = x.shape[0]
        m = sum(p.numel() for p in model.parameters())
        losses = []
        gradients = []
        parameters = []
        noise_para = lipschitz * constraint * math.sqrt(8 * num_iters * math.log(1 / delta)) / (n * epsilon)

        for i in tqdm(range(1, num_iters + 1), desc="Training with privacy"):
            minibatch_indices = np.random.choice(x.shape[0], minibatch_size, replace=True)
            minibatch_x = x[minibatch_indices]
            minibatch_y = y[minibatch_indices]
            loss = model(minibatch_x, minibatch_y)
            loss.backward()
            gradient = torch.cat([p.grad.flatten() for p in model.parameters()]).detach().numpy()
            pos_alphas = self.compute_alpha_private(constraint, gradient, model, noise_para)
            neg_alphas = self.compute_alpha_private(-constraint, gradient, model, noise_para)
            alphas = pos_alphas + neg_alphas
            min_alpha, size, corner_num = min(alphas, key=lambda x: x[0])
            corner = np.zeros(m)
            corner[corner_num] = size
            mu = 2 / (i + 2)
            with torch.no_grad():
                index = 0
                for p in model.parameters():
                    numel = p.numel()
                    p_flat = p.view(-1)
                    p_flat.copy_((1 - mu) * p_flat + mu * torch.tensor(corner[index:index+numel], dtype=torch.float32))
                    index += numel
            losses.append(loss.item())
            gradients.append(np.linalg.norm(gradient))
            parameters.append(model.state_dict().copy())
            model.zero_grad()

        min_loss_index = np.argmin(losses)
        model.load_state_dict(parameters[min_loss_index])

        return model, mean, std, losses[-1], self.calculate_mse(model, x, y)

    def compute_alpha(self, corner_size, gradient, model):
        alpha = gradient * corner_size
        corner_size = (np.ones(sum(p.numel() for p in model.parameters())) * corner_size).tolist()
        corner_num = np.arange(sum(p.numel() for p in model.parameters())).tolist()
        return list(zip(alpha, corner_size, corner_num))

    def compute_alpha_private(self, corner_size, gradient, model, noise_para):
        alpha = gradient * corner_size
        noise = np.random.laplace(scale=noise_para, size=sum(p.numel() for p in model.parameters()))
        alpha = alpha + noise
        corner_size = (np.ones(sum(p.numel() for p in model.parameters())) * corner_size).tolist()
        corner_num = np.arange(sum(p.numel() for p in model.parameters())).tolist()
        return list(zip(alpha, corner_size, corner_num))

    def calculate_mse(self, model, x, y):
        with torch.no_grad():
            predictions = model.predict(x)
            mse = torch.mean((predictions - y) ** 2).item()
        return mse

    def run_experiments_parallel_fixed_hyperparams(self, N_values, epsilon_values, num_repeats=50, constraint=100, minibatch_size=50, lipschitz=1):
        results = []
        for N in N_values:
            x, y = self.generate_data(N)
            x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
            args_list = [(N, epsilon, method, x_train, y_train, x_test, y_test, constraint, minibatch_size, lipschitz) 
                         for epsilon in epsilon_values 
                         for method in ['mle', 'without_private', 'private'] 
                         for _ in range(num_repeats)]
            
            with ProcessPoolExecutor() as executor:
                for result in executor.map(self.run_experiment_fixed_hyperparams, args_list):
                    results.append(result)
        
        # 将结果转换为 DataFrame
        df = pd.DataFrame(results)
        
        # 只对 mse 列取平均值
        df_avg = df.groupby(['N', 'epsilon', 'method'], as_index=False)['mse'].mean()
        
        # 保存结果
        df_avg.to_csv('result_epsilon_fixed_hyperparams_avg.csv', index=False)

    def run_experiment_fixed_hyperparams(self, args):
        N, epsilon, method, x_train, y_train, x_test, y_test, constraint, minibatch_size, lipschitz = args
        if method == 'mle':
            beta, lambda_, sigma_sq, mse, loss = self.stochastic_frontier_mle(x_train, y_train)
            y_pred = x_test.dot(beta)
            test_mse = np.mean((y_test - y_pred) ** 2)
            return {
                'N': N,
                'epsilon': epsilon,
                'method': method,
                'mse': test_mse
            }
        elif method == 'without_private':
            model, _, _, loss, mse = self.train_model_without_private(
                x_train, y_train, num_iters=3000, constraint=constraint, 
                minibatch_size=minibatch_size
            )
            with torch.no_grad():
                y_pred = model.predict(torch.tensor(x_test, dtype=torch.float32)).numpy()
                test_mse = np.mean((y_test - y_pred) ** 2)
            return {
                'N': N,
                'epsilon': 0,  # 将 epsilon 设置为 0，表示不随 epsilon 变化
                'method': method,
                'mse': test_mse
            }
        else:  # method == 'private'
            model, _, _, loss, mse = self.train_model_private(
                x_train, y_train, num_iters=3000, constraint=constraint, 
                minibatch_size=minibatch_size, lipschitz=lipschitz, epsilon=epsilon
            )
            with torch.no_grad():
                y_pred = model.predict(torch.tensor(x_test, dtype=torch.float32)).numpy()
                test_mse = np.mean((y_test - y_pred) ** 2)
            return {
                'N': N,
                'epsilon': epsilon,
                'method': method,
                'mse': test_mse
            }

# 示例用法
num_features = 3
beta = np.array([1.0, 2.0, 3.0])
sigma_u = 0.3
sigma_v = 0.5
sfa = StochasticFrontierAnalysis(num_features, beta, sigma_u, sigma_v)

# 固定超参数
constraint = 60
minibatch_size = 50
lipschitz = 0.5

# 运行实验
sfa.run_experiments_parallel_fixed_hyperparams(
    N_values=[5000], 
    epsilon_values=np.linspace(0.1, 1, 20),
    constraint=constraint,
    minibatch_size=minibatch_size,
    lipschitz=lipschitz
)

# 测试

In [1]:
import torch
import torch.nn as nn
import numpy as np
from scipy.stats import halfnorm
import scipy.stats as stats
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy.optimize import minimize
import statsmodels.api as sm
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from concurrent.futures import ProcessPoolExecutor
import seaborn as sns
from tqdm import tqdm
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error

# Stochastic Frontier Model类
class StochasticFrontierModel(nn.Module):
    def __init__(self, num_features):
        super().__init__()
        self.beta = nn.Parameter(torch.tensor(num_features * [1.0], dtype=torch.float32))
        self.log_sigma2 = nn.Parameter(torch.tensor(0.0))
        self.log_lambda0 = nn.Parameter(torch.tensor(0.0))

    def predict(self, x):
        predictions = torch.matmul(x, self.beta)
        return predictions

    def forward(self, x, y):
        sigma2 = torch.exp(self.log_sigma2)
        lambda0 = torch.exp(self.log_lambda0)
        sigma = torch.sqrt(sigma2)
        epsilon = y - torch.sum(x * self.beta, dim=1)

        term1 = x.shape[0] * self.log_sigma2 / 2
        term2 = -torch.sum(torch.log(torch.distributions.normal.Normal(0, 1).cdf(-epsilon * lambda0 / sigma) + 1e-10))
        term3 = torch.sum(epsilon**2) / (2 * sigma2)

        return (term1 + term2 + term3) / x.shape[0]

class StochasticFrontierAnalysis:
    def __init__(self, num_features, beta, sigma_u, sigma_v):
        self.num_features = num_features
        self.beta = beta
        self.sigma_u = sigma_u
        self.sigma_v = sigma_v

    def standardize_data(self, x):
        mean = np.mean(x[:, 1:], axis=0)
        std = np.std(x[:, 1:], axis=0)
        x_standardized = np.column_stack([x[:, 0], (x[:, 1:] - mean) / std])
        return x_standardized, np.insert(mean, 0, 0), np.insert(std, 0, 1)

    def generate_data(self, N):
        x1 = np.random.uniform(0, 1, N)
        x_random = np.random.normal(0, 1, (N, self.num_features - 1))
        x = np.hstack([x1.reshape(-1, 1), x_random])
        v = np.random.normal(0, self.sigma_v, N)
        u = stats.halfnorm.rvs(loc=0, scale=self.sigma_u, size=N)
        y = x.dot(self.beta) + v - u
        return x, y

    def stochastic_frontier_mle(self, x, y):
        x_df = pd.DataFrame(x)
        y_series = pd.Series(y)

        def logLikFun(param):
            const = param[0]
            parlab = param[:-2]
            parsigmaSq = param[-2]
            parlambda = param[-1]
            epsilon = y_series - 0 - np.dot(x_df, parlab)
            return -np.sum(0.5 * np.log(parsigmaSq) + 0.5 / parsigmaSq * epsilon**2 -
                           norm.logcdf(-epsilon * parlambda / np.sqrt(parsigmaSq)))

        ols = sm.OLS(y_series, x_df).fit()
        init_params = np.append(ols.params.values, [0.5, sum(ols.resid**2) / (len(y_series) - len(ols.params))])

        result = minimize(lambda params: -logLikFun(params), init_params, method='Nelder-Mead')

        beta = result.x[:-2]
        sigma_sq = result.x[-2]
        lambda_ = result.x[-1]

        epsilon = y_series - np.dot(x_df, beta)
        mse = np.mean(epsilon**2)

        return beta, lambda_, sigma_sq, mse, -logLikFun(result.x)

    def adaptive_lasso(self, x, y, beta_true):
        lasso = Lasso(alpha=0.1)
        lasso.fit(x, y)
        weights = 1 / np.abs(lasso.coef_)
        weights[np.isinf(weights)] = 1e10  # 防止除零
        adaptive_lasso = Lasso(alpha=0.1)
        adaptive_lasso.fit(x, y, sample_weight=weights)
        beta_est = adaptive_lasso.coef_
        fr = np.sum((beta_est != 0) & (beta_true == 0)) / np.sum(beta_true == 0)  # False Recovery
        return beta_est, fr

    def train_model_without_private(self, x, y, num_iters=3000, constraint=100, minibatch_size=50):
        x, mean, std = self.standardize_data(x)
        x = torch.tensor(x, dtype=torch.float32)
        y = torch.tensor(y, dtype=torch.float32)
        model = StochasticFrontierModel(num_features=self.num_features)
        losses = []
        gradients = []
        parameters = []

        for i in tqdm(range(1, num_iters + 1), desc="Training without privacy"):
            minibatch_indices = np.random.choice(x.shape[0], minibatch_size, replace=True)
            minibatch_x = x[minibatch_indices]
            minibatch_y = y[minibatch_indices]
            loss = model(minibatch_x, minibatch_y)
            loss.backward()
            gradient = torch.cat([p.grad.flatten() for p in model.parameters()]).detach().numpy()
            pos_alphas = self.compute_alpha(constraint, gradient, model)
            neg_alphas = self.compute_alpha(-constraint, gradient, model)
            alphas = pos_alphas + neg_alphas
            min_alpha, size, corner_num = min(alphas, key=lambda x: x[0])
            corner = np.zeros(sum(p.numel() for p in model.parameters()))
            corner[corner_num] = size
            mu = 2 / (i + 2)
            with torch.no_grad():
                index = 0
                for p in model.parameters():
                    numel = p.numel()
                    p_flat = p.view(-1)
                    p_flat.copy_((1 - mu) * p_flat + mu * torch.tensor(corner[index:index+numel], dtype=torch.float32))
                    index += numel
            losses.append(loss.item())
            gradients.append(np.linalg.norm(gradient))
            parameters.append(model.state_dict().copy())
            model.zero_grad()

        min_loss_index = np.argmin(losses)
        model.load_state_dict(parameters[min_loss_index])

        return model, mean, std, losses[-1], self.calculate_mse(model, x, y)

    def train_model_private(self, x, y, num_iters=3000, constraint=100, minibatch_size=50, lipschitz=1, epsilon=0.1, delta=1e-5):
        x, mean, std = self.standardize_data(x)
        x = torch.tensor(x, dtype=torch.float32)
        y = torch.tensor(y, dtype=torch.float32)
        model = StochasticFrontierModel(num_features=self.num_features)
        n = x.shape[0]
        m = sum(p.numel() for p in model.parameters())
        losses = []
        gradients = []
        parameters = []
        noise_para = lipschitz * constraint * math.sqrt(8 * num_iters * math.log(1 / delta)) / (n * epsilon)

        for i in tqdm(range(1, num_iters + 1), desc="Training with privacy"):
            minibatch_indices = np.random.choice(x.shape[0], minibatch_size, replace=True)
            minibatch_x = x[minibatch_indices]
            minibatch_y = y[minibatch_indices]
            loss = model(minibatch_x, minibatch_y)
            loss.backward()
            gradient = torch.cat([p.grad.flatten() for p in model.parameters()]).detach().numpy()
            pos_alphas = self.compute_alpha_private(constraint, gradient, model, noise_para)
            neg_alphas = self.compute_alpha_private(-constraint, gradient, model, noise_para)
            alphas = pos_alphas + neg_alphas
            min_alpha, size, corner_num = min(alphas, key=lambda x: x[0])
            corner = np.zeros(m)
            corner[corner_num] = size
            mu = 2 / (i + 2)
            with torch.no_grad():
                index = 0
                for p in model.parameters():
                    numel = p.numel()
                    p_flat = p.view(-1)
                    p_flat.copy_((1 - mu) * p_flat + mu * torch.tensor(corner[index:index+numel], dtype=torch.float32))
                    index += numel
            losses.append(loss.item())
            gradients.append(np.linalg.norm(gradient))
            parameters.append(model.state_dict().copy())
            model.zero_grad()

        min_loss_index = np.argmin(losses)
        model.load_state_dict(parameters[min_loss_index])

        return model, mean, std, losses[-1], self.calculate_mse(model, x, y)

    def compute_alpha(self, corner_size, gradient, model):
        alpha = gradient * corner_size
        corner_size = (np.ones(sum(p.numel() for p in model.parameters())) * corner_size).tolist()
        corner_num = np.arange(sum(p.numel() for p in model.parameters())).tolist()
        return list(zip(alpha, corner_size, corner_num))

    def compute_alpha_private(self, corner_size, gradient, model, noise_para):
        alpha = gradient * corner_size
        noise = np.random.laplace(scale=noise_para, size=sum(p.numel() for p in model.parameters()))
        alpha = alpha + noise
        corner_size = (np.ones(sum(p.numel() for p in model.parameters())) * corner_size).tolist()
        corner_num = np.arange(sum(p.numel() for p in model.parameters())).tolist()
        return list(zip(alpha, corner_size, corner_num))

    def calculate_mse(self, model, x, y):
        with torch.no_grad():
            predictions = model.predict(x)
            mse = torch.mean((predictions - y) ** 2).item()
        return mse

    def run_experiments_parallel_fixed_hyperparams(self, N_values, epsilon_values, methods, num_repeats=50, constraint=100, minibatch_size=50, lipschitz=1):
        results = []
        for N in N_values:
            x, y = self.generate_data(N)
            x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
            args_list = [(N, epsilon, method, x_train, y_train, x_test, y_test, constraint, minibatch_size, lipschitz) 
                         for epsilon in epsilon_values 
                         for method in methods 
                         for _ in range(num_repeats)]
            
            with ProcessPoolExecutor() as executor:
                for result in executor.map(self.run_experiment_fixed_hyperparams, args_list):
                    results.append(result)
        
        # 将结果转换为 DataFrame
        df = pd.DataFrame(results)
        
        # 只对 mse 列取平均值
        df_avg = df.groupby(['N', 'epsilon', 'method'], as_index=False)['mse'].mean()
        
        # 保存结果
        df_avg.to_csv('result_epsilon_fixed_hyperparams_avg.csv', index=False)

    def run_experiment_fixed_hyperparams(self, args):
        N, epsilon, method, x_train, y_train, x_test, y_test, constraint, minibatch_size, lipschitz = args
        if method == 'mle':
            beta, lambda_, sigma_sq, mse, loss = self.stochastic_frontier_mle(x_train, y_train)
            beta_est, fr = self.adaptive_lasso(x_train, y_train, self.beta)
            y_pred = x_test.dot(beta_est)
            test_mse = np.mean((y_test - y_pred) ** 2)
            return {
                'N': N,
                'epsilon': epsilon,
                'method': method,
                'mse': test_mse,
                'fr': fr
            }
        elif method == 'without_private':
            model, _, _, loss, mse = self.train_model_without_private(
                x_train, y_train, num_iters=3000, constraint=constraint, 
                minibatch_size=minibatch_size
            )
            with torch.no_grad():
                y_pred = model.predict(torch.tensor(x_test, dtype=torch.float32)).numpy()
                test_mse = np.mean((y_test - y_pred) ** 2)
            return {
                'N': N,
                'epsilon': 0,  # 将 epsilon 设置为 0，表示不随 epsilon 变化
                'method': method,
                'mse': test_mse
            }
        else:  # method == 'private'
            model, _, _, loss, mse = self.train_model_private(
                x_train, y_train, num_iters=3000, constraint=constraint, 
                minibatch_size=minibatch_size, lipschitz=lipschitz, epsilon=epsilon
            )
            with torch.no_grad():
                y_pred = model.predict(torch.tensor(x_test, dtype=torch.float32)).numpy()
                test_mse = np.mean((y_test - y_pred) ** 2)
            return {
                'N': N,
                'epsilon': epsilon,
                'method': method,
                'mse': test_mse
            }

# 示例用法
num_features = 100
beta = np.zeros(num_features)
beta[[10, 20, 30, 40, 50]] = [1, -1, 1, -1, 1]  # 只有5个非零值
sigma_u = 0.3
sigma_v = 0.5
sfa = StochasticFrontierAnalysis(num_features, beta, sigma_u, sigma_v)

# 固定超参数
constraint = 60
minibatch_size = 50
lipschitz = 0.5

# 选择运行的方法
#methods = ['mle', 'without_private', 'private']  # 可以选择运行的方法
methods=['mle']
# 运行实验
sfa.run_experiments_parallel_fixed_hyperparams(
    N_values=[5000], 
    epsilon_values=np.linspace(0.1, 1, 20),
    methods=methods,
    constraint=constraint,
    minibatch_size=minibatch_size,
    lipschitz=lipschitz
)

Exception in thread Thread-3:
Traceback (most recent call last):
  File "D:\anaconda3\envs\pytorch\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "D:\anaconda3\envs\pytorch\lib\concurrent\futures\process.py", line 323, in run
    self.terminate_broken(cause)
  File "D:\anaconda3\envs\pytorch\lib\concurrent\futures\process.py", line 463, in terminate_broken
    work_item.future.set_exception(bpe)
  File "D:\anaconda3\envs\pytorch\lib\concurrent\futures\_base.py", line 561, in set_exception
    raise InvalidStateError('{}: {!r}'.format(self._state, self))
concurrent.futures._base.InvalidStateError: CANCELLED: <Future at 0x1a782071ea0 state=cancelled>


BrokenProcessPool: A process in the process pool was terminated abruptly while the future was running or pending.

In [ ]:
import torch
import torch.nn as nn
import numpy as np
from scipy.stats import halfnorm
import scipy.stats as stats
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy.optimize import minimize
import statsmodels.api as sm
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor, as_completed
import seaborn as sns
from tqdm import tqdm
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error
import logging

# Set up logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

# Stochastic Frontier Model类
class StochasticFrontierModel(nn.Module):
    def __init__(self, num_features):
        super().__init__()
        self.beta = nn.Parameter(torch.tensor(num_features * [1.0], dtype=torch.float32))
        self.log_sigma2 = nn.Parameter(torch.tensor(0.0))
        self.log_lambda0 = nn.Parameter(torch.tensor(0.0))

    def predict(self, x):
        predictions = torch.matmul(x, self.beta)
        return predictions

    def forward(self, x, y):
        sigma2 = torch.exp(self.log_sigma2)
        lambda0 = torch.exp(self.log_lambda0)
        sigma = torch.sqrt(sigma2)
        epsilon = y - torch.sum(x * self.beta, dim=1)

        term1 = x.shape[0] * self.log_sigma2 / 2
        term2 = -torch.sum(torch.log(torch.distributions.normal.Normal(0, 1).cdf(-epsilon * lambda0 / sigma) + 1e-10))
        term3 = torch.sum(epsilon**2) / (2 * sigma2)

        return (term1 + term2 + term3) / x.shape[0]

class StochasticFrontierAnalysis:
    def __init__(self, num_features, beta, sigma_u, sigma_v):
        self.num_features = num_features
        self.beta = beta
        self.sigma_u = sigma_u
        self.sigma_v = sigma_v

    def standardize_data(self, x):
        mean = np.mean(x[:, 1:], axis=0)
        std = np.std(x[:, 1:], axis=0)
        x_standardized = np.column_stack([x[:, 0], (x[:, 1:] - mean) / std])
        return x_standardized, np.insert(mean, 0, 0), np.insert(std, 0, 1)

    def generate_data(self, N):
        x1 = np.random.uniform(0, 1, N)
        x_random = np.random.normal(0, 1, (N, self.num_features - 1))
        x = np.hstack([x1.reshape(-1, 1), x_random])
        v = np.random.normal(0, self.sigma_v, N)
        u = stats.halfnorm.rvs(loc=0, scale=self.sigma_u, size=N)
        y = x.dot(self.beta) + v - u
        return x, y

    def stochastic_frontier_mle(self, x, y):
        x_df = pd.DataFrame(x)
        y_series = pd.Series(y)

        def logLikFun(param):
            const = param[0]
            parlab = param[:-2]
            parsigmaSq = param[-2]
            parlambda = param[-1]
            epsilon = y_series - 0 - np.dot(x_df, parlab)
            return -np.sum(0.5 * np.log(parsigmaSq) + 0.5 / parsigmaSq * epsilon**2 -
                           norm.logcdf(-epsilon * parlambda / np.sqrt(parsigmaSq)))

        ols = sm.OLS(y_series, x_df).fit()
        init_params = np.append(ols.params.values, [0.5, sum(ols.resid**2) / (len(y_series) - len(ols.params))])

        result = minimize(lambda params: -logLikFun(params), init_params, method='Nelder-Mead')

        beta = result.x[:-2]
        sigma_sq = result.x[-2]
        lambda_ = result.x[-1]

        epsilon = y_series - np.dot(x_df, beta)
        mse = np.mean(epsilon**2)

        return beta, lambda_, sigma_sq, mse, -logLikFun(result.x)

    def adaptive_lasso(self, x, y, beta_true):
        lasso = Lasso(alpha=0.1)
        lasso.fit(x, y)
        weights = 1 / np.abs(lasso.coef_)
        weights[np.isinf(weights)] = 1e10  # 防止除零
        adaptive_lasso = Lasso(alpha=0.1)
        adaptive_lasso.fit(x, y, sample_weight=weights)
        beta_est = adaptive_lasso.coef_
        fr = np.sum((beta_est != 0) & (beta_true == 0)) / np.sum(beta_true == 0)  # False Recovery
        return beta_est, fr

    def train_model_without_private(self, x, y, num_iters=3000, constraint=100, minibatch_size=50):
        x, mean, std = self.standardize_data(x)
        x = torch.tensor(x, dtype=torch.float32)
        y = torch.tensor(y, dtype=torch.float32)
        model = StochasticFrontierModel(num_features=self.num_features)
        losses = []
        gradients = []
        parameters = []

        for i in tqdm(range(1, num_iters + 1), desc="Training without privacy"):
            minibatch_indices = np.random.choice(x.shape[0], minibatch_size, replace=True)
            minibatch_x = x[minibatch_indices]
            minibatch_y = y[minibatch_indices]
            loss = model(minibatch_x, minibatch_y)
            loss.backward()
            gradient = torch.cat([p.grad.flatten() for p in model.parameters()]).detach().numpy()
            pos_alphas = self.compute_alpha(constraint, gradient, model)
            neg_alphas = self.compute_alpha(-constraint, gradient, model)
            alphas = pos_alphas + neg_alphas
            min_alpha, size, corner_num = min(alphas, key=lambda x: x[0])
            corner = np.zeros(sum(p.numel() for p in model.parameters()))
            corner[corner_num] = size
            mu = 2 / (i + 2)
            with torch.no_grad():
                index = 0
                for p in model.parameters():
                    numel = p.numel()
                    p_flat = p.view(-1)
                    p_flat.copy_((1 - mu) * p_flat + mu * torch.tensor(corner[index:index+numel], dtype=torch.float32))
                    index += numel
            losses.append(loss.item())
            gradients.append(np.linalg.norm(gradient))
            parameters.append(model.state_dict().copy())
            model.zero_grad()

        min_loss_index = np.argmin(losses)
        model.load_state_dict(parameters[min_loss_index])

        return model, mean, std, losses[-1], self.calculate_mse(model, x, y)

    def train_model_private(self, x, y, num_iters=3000, constraint=100, minibatch_size=50, lipschitz=1, epsilon=0.1, delta=1e-5):
        x, mean, std = self.standardize_data(x)
        x = torch.tensor(x, dtype=torch.float32)
        y = torch.tensor(y, dtype=torch.float32)
        model = StochasticFrontierModel(num_features=self.num_features)
        n = x.shape[0]
        m = sum(p.numel() for p in model.parameters())
        losses = []
        gradients = []
        parameters = []
        noise_para = lipschitz * constraint * math.sqrt(8 * num_iters * math.log(1 / delta)) / (n * epsilon)

        for i in tqdm(range(1, num_iters + 1), desc="Training with privacy"):
            minibatch_indices = np.random.choice(x.shape[0], minibatch_size, replace=True)
            minibatch_x = x[minibatch_indices]
            minibatch_y = y[minibatch_indices]
            loss = model(minibatch_x, minibatch_y)
            loss.backward()
            gradient = torch.cat([p.grad.flatten() for p in model.parameters()]).detach().numpy()
            pos_alphas = self.compute_alpha_private(constraint, gradient, model, noise_para)
            neg_alphas = self.compute_alpha_private(-constraint, gradient, model, noise_para)
            alphas = pos_alphas + neg_alphas
            min_alpha, size, corner_num = min(alphas, key=lambda x: x[0])
            corner = np.zeros(m)
            corner[corner_num] = size
            mu = 2 / (i + 2)
            with torch.no_grad():
                index = 0
                for p in model.parameters():
                    numel = p.numel()
                    p_flat = p.view(-1)
                    p_flat.copy_((1 - mu) * p_flat + mu * torch.tensor(corner[index:index+numel], dtype=torch.float32))
                    index += numel
            losses.append(loss.item())
            gradients.append(np.linalg.norm(gradient))
            parameters.append(model.state_dict().copy())
            model.zero_grad()

        min_loss_index = np.argmin(losses)
        model.load_state_dict(parameters[min_loss_index])

        return model, mean, std, losses[-1], self.calculate_mse(model, x, y)

    def compute_alpha(self, corner_size, gradient, model):
        alpha = gradient * corner_size
        corner_size = (np.ones(sum(p.numel() for p in model.parameters())) * corner_size).tolist()
        corner_num = np.arange(sum(p.numel() for p in model.parameters())).tolist()
        return list(zip(alpha, corner_size, corner_num))

    def compute_alpha_private(self, corner_size, gradient, model, noise_para):
        alpha = gradient * corner_size
        noise = np.random.laplace(scale=noise_para, size=sum(p.numel() for p in model.parameters()))
        alpha = alpha + noise
        corner_size = (np.ones(sum(p.numel() for p in model.parameters())) * corner_size).tolist()
        corner_num = np.arange(sum(p.numel() for p in model.parameters())).tolist()
        return list(zip(alpha, corner_size, corner_num))

    def calculate_mse(self, model, x, y):
        with torch.no_grad():
            predictions = model.predict(x)
            mse = torch.mean((predictions - y) ** 2).item()
        return mse

    def run_experiments_parallel_fixed_hyperparams(self, N_values, epsilon_values, methods, num_repeats=50, constraint=100, minibatch_size=50, lipschitz=1):
        results = []
        for N in N_values:
            x, y = self.generate_data(N)
            x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
            args_list = [(N, epsilon, method, x_train, y_train, x_test, y_test, constraint, minibatch_size, lipschitz) 
                         for epsilon in epsilon_values 
                         for method in methods 
                         for _ in range(num_repeats)]
            
            # Use ThreadPoolExecutor instead of ProcessPoolExecutor for debugging
            with ThreadPoolExecutor() as executor:
                futures = [executor.submit(self.run_experiment_fixed_hyperparams, args) for args in args_list]
                for future in tqdm(as_completed(futures), total=len(futures), desc="Running experiments"):
                    try:
                        result = future.result()
                        results.append(result)
                    except Exception as e:
                        logging.error(f"Error in experiment: {e}")
        
        # 将结果转换为 DataFrame
        df = pd.DataFrame(results)
        
        # 只对 mse 列取平均值
        df_avg = df.groupby(['N', 'epsilon', 'method'], as_index=False)['mse'].mean()
        
        # 保存结果
        df_avg.to_csv('result_epsilon_fixed_hyperparams_avg.csv', index=False)

    def run_experiment_fixed_hyperparams(self, args):
        N, epsilon, method, x_train, y_train, x_test, y_test, constraint, minibatch_size, lipschitz = args
        try:
            if method == 'mle':
                beta, lambda_, sigma_sq, mse, loss = self.stochastic_frontier_mle(x_train, y_train)
                beta_est, fr = self.adaptive_lasso(x_train, y_train, self.beta)
                y_pred = x_test.dot(beta_est)
                test_mse = np.mean((y_test - y_pred) ** 2)
                return {
                    'N': N,
                    'epsilon': epsilon,
                    'method': method,
                    'mse': test_mse,
                    'fr': fr
                }
            elif method == 'without_private':
                model, _, _, loss, mse = self.train_model_without_private(
                    x_train, y_train, num_iters=3000, constraint=constraint, 
                    minibatch_size=minibatch_size
                )
                with torch.no_grad():
                    y_pred = model.predict(torch.tensor(x_test, dtype=torch.float32)).numpy()
                    test_mse = np.mean((y_test - y_pred) ** 2)
                return {
                    'N': N,
                    'epsilon': 0,  # 将 epsilon 设置为 0，表示不随 epsilon 变化
                    'method': method,
                    'mse': test_mse
                }
            else:  # method == 'private'
                model, _, _, loss, mse = self.train_model_private(
                    x_train, y_train, num_iters=3000, constraint=constraint, 
                    minibatch_size=minibatch_size, lipschitz=lipschitz, epsilon=epsilon
                )
                with torch.no_grad():
                    y_pred = model.predict(torch.tensor(x_test, dtype=torch.float32)).numpy()
                    test_mse = np.mean((y_test - y_pred) ** 2)
                return {
                    'N': N,
                    'epsilon': epsilon,
                    'method': method,
                    'mse': test_mse
                }
        except Exception as e:
            logging.error(f"Error in method {method}: {e}")
            return {
                'N': N,
                'epsilon': epsilon,
                'method': method,
                'mse': np.nan,
                'fr': np.nan
            }

# 示例用法
if __name__ == "__main__":
    num_features = 100
    beta = np.zeros(num_features)
    beta[[10, 20, 30, 40, 50]] = [1, -1, 1, -1, 1]  # 只有5个非零值
    sigma_u = 0.3
    sigma_v = 0.5
    sfa = StochasticFrontierAnalysis(num_features, beta, sigma_u, sigma_v)

    # 固定超参数
    constraint = 60
    minibatch_size = 50
    lipschitz = 0.5

    # 选择运行的方法
    methods = ['mle']  # 可以选择运行的方法

    # 运行实验
    sfa.run_experiments_parallel_fixed_hyperparams(
        N_values=[5000], 
        epsilon_values=np.linspace(0.1, 1, 20),
        methods=methods,
        constraint=constraint,
        minibatch_size=minibatch_size,
        lipschitz=lipschitz
    )

Running experiments:   0%|                                                                     | 0/1000 [00:00<?, ?it/s]C:\Users\86177\AppData\Local\Temp\ipykernel_26292\3614381226.py:99: RuntimeWarning: divide by zero encountered in divide
  weights = 1 / np.abs(lasso.coef_)
2025-01-12 10:32:18,997 - ERROR - Error in method mle: sample_weight.shape == (100,), expected (4000,)!
Running experiments:   0%|                                                       | 1/1000 [17:17<287:59:50, 1037.83s/it]C:\Users\86177\AppData\Local\Temp\ipykernel_26292\3614381226.py:99: RuntimeWarning: divide by zero encountered in divide
  weights = 1 / np.abs(lasso.coef_)
2025-01-12 10:32:20,206 - ERROR - Error in method mle: sample_weight.shape == (100,), expected (4000,)!
Running experiments:   0%|                                                        | 2/1000 [17:19<118:40:24, 428.08s/it]C:\Users\86177\AppData\Local\Temp\ipykernel_26292\3614381226.py:99: RuntimeWarning: divide by zero encountered in divi

Running experiments:   2%|█▏                                                      | 22/1000 [34:40<58:48:48, 216.49s/it]C:\Users\86177\AppData\Local\Temp\ipykernel_26292\3614381226.py:99: RuntimeWarning: divide by zero encountered in divide
  weights = 1 / np.abs(lasso.coef_)
2025-01-12 10:49:42,163 - ERROR - Error in method mle: sample_weight.shape == (100,), expected (4000,)!
Running experiments:   2%|█▎                                                      | 23/1000 [34:40<41:08:42, 151.61s/it]C:\Users\86177\AppData\Local\Temp\ipykernel_26292\3614381226.py:99: RuntimeWarning: divide by zero encountered in divide
  weights = 1 / np.abs(lasso.coef_)
2025-01-12 10:49:42,365 - ERROR - Error in method mle: sample_weight.shape == (100,), expected (4000,)!
Running experiments:   2%|█▎                                                      | 24/1000 [34:41<28:47:10, 106.18s/it]C:\Users\86177\AppData\Local\Temp\ipykernel_26292\3614381226.py:99: RuntimeWarning: divide by zero encountered in divi

Running experiments:   4%|██▌                                                      | 44/1000 [52:05<26:23:18, 99.37s/it]C:\Users\86177\AppData\Local\Temp\ipykernel_26292\3614381226.py:99: RuntimeWarning: divide by zero encountered in divide
  weights = 1 / np.abs(lasso.coef_)
2025-01-12 11:07:07,304 - ERROR - Error in method mle: sample_weight.shape == (100,), expected (4000,)!
Running experiments:   4%|██▌                                                      | 45/1000 [52:06<18:53:56, 71.24s/it]C:\Users\86177\AppData\Local\Temp\ipykernel_26292\3614381226.py:99: RuntimeWarning: divide by zero encountered in divide
  weights = 1 / np.abs(lasso.coef_)
2025-01-12 11:07:07,541 - ERROR - Error in method mle: sample_weight.shape == (100,), expected (4000,)!
Running experiments:   5%|██▌                                                      | 46/1000 [52:06<13:25:38, 50.67s/it]C:\Users\86177\AppData\Local\Temp\ipykernel_26292\3614381226.py:99: RuntimeWarning: divide by zero encountered in divi

Running experiments:   7%|███▋                                                   | 66/1000 [1:09:40<13:51:48, 53.44s/it]C:\Users\86177\AppData\Local\Temp\ipykernel_26292\3614381226.py:99: RuntimeWarning: divide by zero encountered in divide
  weights = 1 / np.abs(lasso.coef_)
2025-01-12 11:24:41,796 - ERROR - Error in method mle: sample_weight.shape == (100,), expected (4000,)!
Running experiments:   7%|███▊                                                    | 67/1000 [1:09:40<9:43:14, 37.51s/it]C:\Users\86177\AppData\Local\Temp\ipykernel_26292\3614381226.py:99: RuntimeWarning: divide by zero encountered in divide
  weights = 1 / np.abs(lasso.coef_)
2025-01-12 11:24:44,717 - ERROR - Error in method mle: sample_weight.shape == (100,), expected (4000,)!
Running experiments:   7%|███▊                                                    | 68/1000 [1:09:43<7:01:22, 27.13s/it]C:\Users\86177\AppData\Local\Temp\ipykernel_26292\3614381226.py:99: RuntimeWarning: divide by zero encountered in divi

Running experiments:   9%|████▉                                                   | 88/1000 [1:27:07<7:05:51, 28.02s/it]C:\Users\86177\AppData\Local\Temp\ipykernel_26292\3614381226.py:99: RuntimeWarning: divide by zero encountered in divide
  weights = 1 / np.abs(lasso.coef_)
2025-01-12 11:42:11,232 - ERROR - Error in method mle: sample_weight.shape == (100,), expected (4000,)!
Running experiments:   9%|████▉                                                   | 89/1000 [1:27:10<5:10:52, 20.47s/it]C:\Users\86177\AppData\Local\Temp\ipykernel_26292\3614381226.py:99: RuntimeWarning: divide by zero encountered in divide
  weights = 1 / np.abs(lasso.coef_)
2025-01-12 11:42:14,064 - ERROR - Error in method mle: sample_weight.shape == (100,), expected (4000,)!
Running experiments:   9%|█████                                                   | 90/1000 [1:27:12<3:50:20, 15.19s/it]C:\Users\86177\AppData\Local\Temp\ipykernel_26292\3614381226.py:99: RuntimeWarning: divide by zero encountered in divi

...

Running experiments:  86%|████████████████████████████████████████████████        | 858/1000 [12:27:54<50:27, 21.32s/it]C:\Users\86177\AppData\Local\Temp\ipykernel_26292\3614381226.py:99: RuntimeWarning: divide by zero encountered in divide
  weights = 1 / np.abs(lasso.coef_)
2025-01-12 22:43:35,926 - ERROR - Error in method mle: sample_weight.shape == (100,), expected (4000,)!
Running experiments:  86%|██████████████████████████████████████████████▍       | 859/1000 [12:28:34<1:03:20, 26.96s/it]C:\Users\86177\AppData\Local\Temp\ipykernel_26292\3614381226.py:99: RuntimeWarning: divide by zero encountered in divide
  weights = 1 / np.abs(lasso.coef_)
2025-01-12 22:45:34,815 - ERROR - Error in method mle: sample_weight.shape == (100,), expected (4000,)!
Running experiments:  86%|██████████████████████████████████████████████▍       | 860/1000 [12:30:33<2:07:15, 54.54s/it]C:\Users\86177\AppData\Local\Temp\ipykernel_26292\3614381226.py:99: RuntimeWarning: divide by zero encountered in divi

Running experiments:  88%|███████████████████████████████████████████████▌      | 880/1000 [12:48:13<1:58:21, 59.18s/it]C:\Users\86177\AppData\Local\Temp\ipykernel_26292\3614381226.py:99: RuntimeWarning: divide by zero encountered in divide
  weights = 1 / np.abs(lasso.coef_)
2025-01-12 23:15:13,448 - ERROR - Error in method mle: sample_weight.shape == (100,), expected (4000,)!
Running experiments:  88%|██████████████████████████████████████████████▋      | 881/1000 [13:00:12<8:29:48, 257.05s/it]C:\Users\86177\AppData\Local\Temp\ipykernel_26292\3614381226.py:99: RuntimeWarning: divide by zero encountered in divide
  weights = 1 / np.abs(lasso.coef_)
2025-01-12 23:15:21,158 - ERROR - Error in method mle: sample_weight.shape == (100,), expected (4000,)!
Running experiments:  88%|██████████████████████████████████████████████▋      | 882/1000 [13:00:19<5:58:25, 182.25s/it]C:\Users\86177\AppData\Local\Temp\ipykernel_26292\3614381226.py:99: RuntimeWarning: divide by zero encountered in divi

Running experiments:  90%|███████████████████████████████████████████████▊     | 902/1000 [13:22:23<5:06:23, 187.59s/it]C:\Users\86177\AppData\Local\Temp\ipykernel_26292\3614381226.py:99: RuntimeWarning: divide by zero encountered in divide
  weights = 1 / np.abs(lasso.coef_)
2025-01-12 23:37:47,001 - ERROR - Error in method mle: sample_weight.shape == (100,), expected (4000,)!
Running experiments:  90%|███████████████████████████████████████████████▊     | 903/1000 [13:22:45<3:43:12, 138.06s/it]C:\Users\86177\AppData\Local\Temp\ipykernel_26292\3614381226.py:99: RuntimeWarning: divide by zero encountered in divide
  weights = 1 / np.abs(lasso.coef_)
2025-01-12 23:37:50,923 - ERROR - Error in method mle: sample_weight.shape == (100,), expected (4000,)!
Running experiments:  90%|████████████████████████████████████████████████▊     | 904/1000 [13:22:49<2:36:31, 97.83s/it]C:\Users\86177\AppData\Local\Temp\ipykernel_26292\3614381226.py:99: RuntimeWarning: divide by zero encountered in divi

Running experiments:  92%|█████████████████████████████████████████████████▉    | 924/1000 [13:43:47<1:56:42, 92.14s/it]C:\Users\86177\AppData\Local\Temp\ipykernel_26292\3614381226.py:99: RuntimeWarning: divide by zero encountered in divide
  weights = 1 / np.abs(lasso.coef_)
2025-01-12 23:59:02,326 - ERROR - Error in method mle: sample_weight.shape == (100,), expected (4000,)!
Running experiments:  92%|█████████████████████████████████████████████████▉    | 925/1000 [13:44:01<1:25:54, 68.73s/it]C:\Users\86177\AppData\Local\Temp\ipykernel_26292\3614381226.py:99: RuntimeWarning: divide by zero encountered in divide
  weights = 1 / np.abs(lasso.coef_)
2025-01-12 23:59:09,411 - ERROR - Error in method mle: sample_weight.shape == (100,), expected (4000,)!
Running experiments:  93%|██████████████████████████████████████████████████    | 926/1000 [13:44:08<1:01:57, 50.24s/it]C:\Users\86177\AppData\Local\Temp\ipykernel_26292\3614381226.py:99: RuntimeWarning: divide by zero encountered in divi

Running experiments:  95%|████████████████████████████████████████████████████▉   | 946/1000 [14:05:05<43:18, 48.13s/it]C:\Users\86177\AppData\Local\Temp\ipykernel_26292\3614381226.py:99: RuntimeWarning: divide by zero encountered in divide
  weights = 1 / np.abs(lasso.coef_)
2025-01-13 00:21:21,094 - ERROR - Error in method mle: sample_weight.shape == (100,), expected (4000,)!
Running experiments:  95%|█████████████████████████████████████████████████████   | 947/1000 [14:06:19<49:32, 56.08s/it]C:\Users\86177\AppData\Local\Temp\ipykernel_26292\3614381226.py:99: RuntimeWarning: divide by zero encountered in divide
  weights = 1 / np.abs(lasso.coef_)
2025-01-13 00:21:53,832 - ERROR - Error in method mle: sample_weight.shape == (100,), expected (4000,)!
Running experiments:  95%|█████████████████████████████████████████████████████   | 948/1000 [14:06:52<42:31, 49.08s/it]C:\Users\86177\AppData\Local\Temp\ipykernel_26292\3614381226.py:99: RuntimeWarning: divide by zero encountered in divi

In [ ]:
import torch
import torch.nn as nn
import numpy as np
from scipy.stats import halfnorm
import scipy.stats as stats
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy.optimize import minimize
import statsmodels.api as sm
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from concurrent.futures import ProcessPoolExecutor
import seaborn as sns
from tqdm import tqdm

# Stochastic Frontier Model类
class StochasticFrontierModel(nn.Module):
    def __init__(self, num_features):
        super().__init__()
        self.beta = nn.Parameter(torch.tensor(num_features * [1.0], dtype=torch.float32))
        self.log_sigma2 = nn.Parameter(torch.tensor(0.0))
        self.log_lambda0 = nn.Parameter(torch.tensor(0.0))

    def predict(self, x):
        predictions = torch.matmul(x, self.beta)
        return predictions

    def forward(self, x, y,lambda_reg=1):
        sigma2 = torch.exp(self.log_sigma2)
        lambda0 = torch.exp(self.log_lambda0)
        sigma = torch.sqrt(sigma2)
        epsilon = y - torch.sum(x * self.beta, dim=1)

        term1 = x.shape[0] * self.log_sigma2 / 2
        term2 = -torch.sum(torch.log(torch.distributions.normal.Normal(0, 1).cdf(-epsilon * lambda0 / sigma) + 1e-10))
        term3 = torch.sum(epsilon**2) / (2 * sigma2)
        lasso_penalty = lambda_reg * torch.sum(torch.abs(self.beta))
        return (term1 + term2 + term3+lasso_penalty) / x.shape[0]

class StochasticFrontierAnalysis:
    def __init__(self, num_features, beta, sigma_u, sigma_v):
        self.num_features = num_features
        self.beta = beta
        self.sigma_u = sigma_u
        self.sigma_v = sigma_v

    def standardize_data(self, x):
        mean = np.mean(x[:, 1:], axis=0)
        std = np.std(x[:, 1:], axis=0)
        x_standardized = np.column_stack([x[:, 0], (x[:, 1:] - mean) / std])
        return x_standardized, np.insert(mean, 0, 0), np.insert(std, 0, 1)
    

    def train_model_without_private(self, x, y, num_iters=3000, constraint=100, minibatch_size=50,lambda_reg=1):
        x, mean, std = self.standardize_data(x)
        x = torch.tensor(x, dtype=torch.float32)
        y = torch.tensor(y, dtype=torch.float32)
        model = StochasticFrontierModel(num_features=self.num_features)
        losses = []
        gradients = []
        parameters = []

        for i in tqdm(range(1, num_iters + 1), desc="Training without privacy"):
            minibatch_indices = np.random.choice(x.shape[0], minibatch_size, replace=True)
            minibatch_x = x[minibatch_indices]
            minibatch_y = y[minibatch_indices]
            loss = model(minibatch_x, minibatch_y,lambda_reg)
            loss.backward()
            gradient = torch.cat([p.grad.flatten() for p in model.parameters()]).detach().numpy()
            pos_alphas = self.compute_alpha(constraint, gradient, model)
            neg_alphas = self.compute_alpha(-constraint, gradient, model)
            alphas = pos_alphas + neg_alphas
            min_alpha, size, corner_num = min(alphas, key=lambda x: x[0])
            corner = np.zeros(sum(p.numel() for p in model.parameters()))
            corner[corner_num] = size
            mu = 2 / (i + 2)
            with torch.no_grad():
                index = 0
                for p in model.parameters():
                    numel = p.numel()
                    p_flat = p.view(-1)
                    p_flat.copy_((1 - mu) * p_flat + mu * torch.tensor(corner[index:index+numel], dtype=torch.float32))
                    index += numel
            losses.append(loss.item())
            gradients.append(np.linalg.norm(gradient))
            parameters.append(model.state_dict().copy())
            model.zero_grad()

        min_loss_index = np.argmin(losses)
        model.load_state_dict(parameters[min_loss_index])

        return model, mean, std, losses[-1], self.calculate_mse(model, x, y)

    def train_model_private(self, x, y, num_iters=3000, constraint=100, minibatch_size=50, lipschitz=1, epsilon=0.1, delta=1e-5,lamda_reg=1):
        x, mean, std = self.standardize_data(x)
        x = torch.tensor(x, dtype=torch.float32)
        y = torch.tensor(y, dtype=torch.float32)
        model = StochasticFrontierModel(num_features=self.num_features)
        n = x.shape[0]
        m = sum(p.numel() for p in model.parameters())
        losses = []
        gradients = []
        parameters = []
        noise_para = lipschitz * constraint * math.sqrt(8 * num_iters * math.log(1 / delta)) / (n * epsilon)

        for i in tqdm(range(1, num_iters + 1), desc="Training with privacy"):
            minibatch_indices = np.random.choice(x.shape[0], minibatch_size, replace=True)
            minibatch_x = x[minibatch_indices]
            minibatch_y = y[minibatch_indices]
            loss = model(minibatch_x, minibatch_y,lambda_reg)
            loss.backward()
            gradient = torch.cat([p.grad.flatten() for p in model.parameters()]).detach().numpy()
            pos_alphas = self.compute_alpha_private(constraint, gradient, model, noise_para)
            neg_alphas = self.compute_alpha_private(-constraint, gradient, model, noise_para)
            alphas = pos_alphas + neg_alphas
            min_alpha, size, corner_num = min(alphas, key=lambda x: x[0])
            corner = np.zeros(m)
            corner[corner_num] = size
            mu = 2 / (i + 2)
            with torch.no_grad():
                index = 0
                for p in model.parameters():
                    numel = p.numel()
                    p_flat = p.view(-1)
                    p_flat.copy_((1 - mu) * p_flat + mu * torch.tensor(corner[index:index+numel], dtype=torch.float32))
                    index += numel
            losses.append(loss.item())
            gradients.append(np.linalg.norm(gradient))
            parameters.append(model.state_dict().copy())
            model.zero_grad()

        min_loss_index = np.argmin(losses)
        model.load_state_dict(parameters[min_loss_index])

        return model, mean, std, losses[-1], self.calculate_mse(model, x, y)

    def compute_alpha(self, corner_size, gradient, model):
        alpha = gradient * corner_size
        corner_size = (np.ones(sum(p.numel() for p in model.parameters())) * corner_size).tolist()
        corner_num = np.arange(sum(p.numel() for p in model.parameters())).tolist()
        return list(zip(alpha, corner_size, corner_num))

    def compute_alpha_private(self, corner_size, gradient, model, noise_para):
        alpha = gradient * corner_size
        noise = np.random.laplace(scale=noise_para, size=sum(p.numel() for p in model.parameters()))
        alpha = alpha + noise
        corner_size = (np.ones(sum(p.numel() for p in model.parameters())) * corner_size).tolist()
        corner_num = np.arange(sum(p.numel() for p in model.parameters())).tolist()
        return list(zip(alpha, corner_size, corner_num))

    def calculate_mse(self, model, x, y):
        with torch.no_grad():
            predictions = model.predict(x)
            mse = torch.mean((predictions - y) ** 2).item()
        return mse

    
    def run_experiment_fixed_hyperparams(self, args):
        N, epsilon, method, x_train, y_train, x_test, y_test, constraint, minibatch_size, lipschitz = args
        if method == 'mle':
            beta, lambda_, sigma_sq, mse, loss = self.stochastic_frontier_mle(x_train, y_train)
            y_pred = x_test.dot(beta)
            test_mse = np.mean((y_test - y_pred) ** 2)
            return {
                'N': N,
                'epsilon': epsilon,
                'method': method,
                'mse': test_mse
            }
        elif method == 'without_private':
            model, _, _, loss, mse = self.train_model_without_private(
                x_train, y_train, num_iters=3000, constraint=constraint, 
                minibatch_size=minibatch_size
            )
            with torch.no_grad():
                y_pred = model.predict(torch.tensor(x_test, dtype=torch.float32)).numpy()
                test_mse = np.mean((y_test - y_pred) ** 2)
            return {
                'N': N,
                'epsilon': 0,  # 将 epsilon 设置为 0，表示不随 epsilon 变化
                'method': method,
                'mse': test_mse
            }
        else:  # method == 'private'
            model, _, _, loss, mse = self.train_model_private(
                x_train, y_train, num_iters=3000, constraint=constraint, 
                minibatch_size=minibatch_size, lipschitz=lipschitz, epsilon=epsilon
            )
            with torch.no_grad():
                y_pred = model.predict(torch.tensor(x_test, dtype=torch.float32)).numpy()
                test_mse = np.mean((y_test - y_pred) ** 2)
            return {
                'N': N,
                'epsilon': epsilon,
                'method': method,
                'mse': test_mse
            }

# 示例用法
num_features = 3
beta = np.array([1.0, 2.0, 3.0])
sigma_u = 0.3
sigma_v = 0.5
sfa = StochasticFrontierAnalysis(num_features, beta, sigma_u, sigma_v)

# 固定超参数
constraint = 60
minibatch_size = 50
lipschitz = 0.5

# 运行实验
sfa.run_experiments_parallel_fixed_hyperparams(
    N_values=[5000], 
    epsilon_values=np.linspace(0.1, 1, 20),
    constraint=constraint,
    minibatch_size=minibatch_size,
    lipschitz=lipschitz
)